In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
import psycopg2 as pg_driver

db = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='jetic900', 
                        host='localhost', 
                        port='5432'
                    )



def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                


In [4]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                        database="postgres", 
                        user='postgres',
                        password='jetic900', 
                        host='localhost', 
                        port='5432'
                    );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [5]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark'),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Dima'),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Phill'),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Viktor'),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Ivan')
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 3),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 2),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 1)
                 
            
        """

execute_query(query)

In [6]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (1, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5)
                 

                 
            
        """

execute_query(query)

In [7]:
all_rows = execute_query("select user_id, count(*) from likes group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=3, count=1)
1 Record(user_id=5, count=1)
2 Record(user_id=4, count=1)
3 Record(user_id=2, count=2)
4 Record(user_id=1, count=3)


In [8]:
all_rows = execute_query("select user_id, count(*) from likes where user_id=1 group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, count=3)


In [9]:
query = """select l.user_id, u.name 
           from likes l
           left join users u
           on l.user_id=u.user_id
           where l.user_id=1 
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Mark')
1 Record(user_id=1, name='Mark')
2 Record(user_id=1, name='Mark')


In [10]:
query = """select
                l.user_id as from_id,
                u.name as from_name,
                l.user_id_give as to_id,
                us.name as to_name
           from likes l
           left join users u
           on l.user_id=u.user_id
           left join users us
           on l.user_id_give=us.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    

0 Record(from_id=5, from_name='Ivan', to_id=1, to_name='Mark')
1 Record(from_id=4, from_name='Viktor', to_id=2, to_name='Dima')
2 Record(from_id=3, from_name='Phill', to_id=3, to_name='Phill')
3 Record(from_id=2, from_name='Dima', to_id=4, to_name='Viktor')
4 Record(from_id=1, from_name='Mark', to_id=4, to_name='Viktor')
5 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')
6 Record(from_id=2, from_name='Dima', to_id=5, to_name='Ivan')
7 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')


In [11]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark')
 
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4)
            
        """

execute_query(query)

In [12]:
query = """select
                l.user_id as from_id,
                u.name as from_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by l.user_id, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    

0 Record(from_id=4, from_name='Viktor', like_count=1)
1 Record(from_id=1, from_name='Mark', like_count=3)
2 Record(from_id=3, from_name='Phill', like_count=1)
3 Record(from_id=10, from_name='Mark', like_count=1)
4 Record(from_id=2, from_name='Dima', like_count=2)
5 Record(from_id=5, from_name='Ivan', like_count=1)


In [13]:
query = """select
                --l.user_id as from_id,
                u.name as from_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_name='Viktor', like_count=1)
1 Record(from_name='Phill', like_count=1)
2 Record(from_name='Ivan', like_count=1)
3 Record(from_name='Mark', like_count=4)
4 Record(from_name='Dima', like_count=2)


In [14]:
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(to_id=4, to_name='Viktor', like_count=3)
1 Record(to_id=1, to_name='Mark', like_count=1)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=5, to_name='Ivan', like_count=3)


In [15]:
query = """select *
                
           from likes l
           order by l.user_id_give
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    

0 Record(user_id=5, created=datetime.datetime(2018, 6, 16, 16, 36, 38), user_id_give=1)
1 Record(user_id=4, created=datetime.datetime(2018, 6, 16, 15, 36, 38), user_id_give=2)
2 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), user_id_give=3)
3 Record(user_id=10, created=datetime.datetime(2021, 5, 16, 15, 36, 38), user_id_give=4)
4 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), user_id_give=4)
5 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), user_id_give=4)
6 Record(user_id=1, created=datetime.datetime(2018, 6, 16, 14, 36, 38), user_id_give=5)
7 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), user_id_give=5)
8 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), user_id_give=5)


In [16]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (8, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Damir')
 
        """

execute_query(query)

In [17]:
query = """select
                
                u.user_id as user_id,
                u.name as user_name,
                l.user_id_give as to_id
                
                
           from users u
           left join likes l
           on u.user_id=l.user_id
           where l.user_id_give is null
           order by u.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=8, user_name='Damir', to_id=None)
